<a href="https://colab.research.google.com/github/yash-clear/Anomaly_Detection/blob/main/ARIMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Time Series Anomaly *Detection*

### ARIMA statistical model as a baseline — classic auto-regression model that is made exactly for the time series

## The Numenta Anomaly Benchmark

### LOADING THE DATASET

In [1]:

%%bash
if [ ! -d "NAB" ]; then
    git clone https://github.com/numenta/NAB
fi

Cloning into 'NAB'...
Checking out files: 100% (1119/1119), done.


In [2]:

from pathlib import Path # convenient way to deal w/ paths
import plotly.graph_objects as go # creates plots
import numpy as np # standard for data processing
import pandas as pd # standard for data processing
import json # we have anomalies' timestamps in json format

In [3]:

# Path to the whole data from NAB git repository
nab = Path.cwd()/'NAB'

# This folder contains all files w/ metrics
data_path = nab/'data'

# There is also separate json file 
# w/ timestamps of anomalies in files w/ metrics
labels_filepath = '/content/NAB/labels/combined_labels.json'

# Path from data folder to the training file
training_filename = 'realAWSCloudwatch/rds_cpu_utilization_cc0c53.csv'

# Path from data folder to the validation file
valid_filename = 'realAWSCloudwatch/rds_cpu_utilization_e47b3b.csv'

In [4]:

with open(labels_filepath, 'r') as f:
    anomalies_timestamps = json.load(f)

In [5]:

train = pd.read_csv('/content/NAB/data/realAWSCloudwatch/rds_cpu_utilization_cc0c53.csv')
valid = pd.read_csv('/content/NAB/data/realAWSCloudwatch/rds_cpu_utilization_e47b3b.csv')
train.head()

,timestamp,value
0,2014-02-14 14:30:00,6.456
1,2014-02-14 14:35:00,5.816
2,2014-02-14 14:40:00,6.268
3,2014-02-14 14:45:00,5.816
4,2014-02-14 14:50:00,5.862


In [6]:
valid.head()

,timestamp,value
0,2014-04-10 00:02:00,14.012
1,2014-04-10 00:07:00,13.334
2,2014-04-10 00:12:00,15.000
3,2014-04-10 00:17:00,13.998
4,2014-04-10 00:22:00,14.332


In [7]:

from sklearn.preprocessing import StandardScaler

# Let's make it function for further usage
def parse_and_standardize(df: pd.DataFrame, scaler: StandardScaler = None):
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['stand_value'] = df['value']
    if not scaler:
        scaler = StandardScaler()
        scaler.fit(df['stand_value'].values.reshape(-1, 1))
    df['stand_value'] = scaler.transform(df['stand_value'].values.reshape(-1, 1))
    return scaler

data_scaler = parse_and_standardize(train)
parse_and_standardize(valid, data_scaler)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [8]:

train_anomalies = train[train['timestamp'].isin(anomalies_timestamps['realAWSCloudwatch/rds_cpu_utilization_cc0c53.csv'])]
valid_anomalies = valid[valid['timestamp'].isin(anomalies_timestamps['realAWSCloudwatch/rds_cpu_utilization_e47b3b.csv'])]
train_anomalies

,timestamp,value,stand_value
3080,2014-02-25 07:15:00,25.1033,4.652449
3579,2014-02-27 00:50:00,19.1650,3.026441


In [9]:
# Prepare layout w/ titles

import plotly.graph_objects as go
layout = dict(xaxis=dict(title='Timestamp'), yaxis=dict(title='CPU Utilization')) 

# Create the figure for plotting the data
fig = go.Figure(layout=layout) 

# Add non-anomaly data to the figure
fig.add_trace(go.Scatter(x=train['timestamp'], y=train['value'], 
                         mode='markers', name='Non-anomaly',
                         marker=dict(color='blue')))

# Add anomaly data to the figure
fig.add_trace(go.Scatter(x=train_anomalies['timestamp'],
                         y=train_anomalies['value'], 
                         mode='markers', name='Anomaly',
                         marker=dict(color='green', size=13)))
fig.update_layout(
    title="Training set"
)
fig.show()

In [10]:

fig = go.Figure()
fig.add_trace(go.Scatter(x=valid['timestamp'], y=valid['value'], 
                         mode='markers', name='Non-anomaly',
                         marker=dict(color='blue')))
fig.add_trace(go.Scatter(x=valid_anomalies['timestamp'],
                         y=valid_anomalies['value'], 
                         mode='markers', name='Anomaly',
                         marker=dict(color='green', size=13)))
fig.update_layout(
    title="Validation set"
    )

fig.show()

In [11]:

import statsmodels.api as sm
from itertools import product

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



### TRAIN THE MODEL

In [12]:
import warnings

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
    # Initial approximation of parameters
Qs = range(0, 2)
qs = range(0, 3)
Ps = range(0, 3)
ps = range(0, 3)
D=1
d=1
parameters = product(ps, qs, Ps, Qs)
parameters_list = list(parameters)

# Best Model Selection
results = []
best_aic = float("inf")
warnings.filterwarnings('ignore')
iterations=1
for param in parameters_list:
    try:
        model=sm.tsa.statespace.SARIMAX(train.value, order=(param[0], d, param[1]),seasonal_order=(param[2], D, param[3], 12),).fit(disp=-1)
        #print("Iteration:",iterations)
    except ValueError:
        print('wrong parameters:', param)
        continue
    aic = model.aic
    if aic < best_aic:
        best_model = model
        best_aic = aic
        best_param = param
    results.append([param, model.aic])

# Writing of the predictions for training data
train['predict'] = best_model.predict()
#train['predict'].fillna(0, inplace=True)

# Writing of the predictions for validation data
best_model_valid = sm.tsa.statespace.SARIMAX(
    valid.value, order=(best_param[0], d, best_param[1]),
    seasonal_order=(best_param[2], D, best_param[3], 12),
    initialization='approximate_diffuse'
    ).fit()
valid['predict'] = best_model_valid.predict()
#valid['predict'].fillna(0, inplace=True)
# Calling of the function



In [13]:

    
fig = go.Figure()
fig.add_trace(go.Scatter(x=valid['timestamp'],
                        y=valid['value'], 
                        mode='markers', name='Ground Truth',
                        marker=dict(color='blue', size=5)))

fig.add_trace(go.Scatter(x=valid['timestamp'], y=valid['predict'], 
                        mode='markers', name='Predicted Value',
                        marker=dict(color='orange')))
fig.update_layout(
    title="Validation set"
    )

fig.show()

In [14]:

    
fig = go.Figure()
fig.add_trace(go.Scatter(x=train['timestamp'],
                        y=train['value'], 
                        mode='markers', name='Ground Truth',
                        marker=dict(color='blue', size=5)))

fig.add_trace(go.Scatter(x=train['timestamp'], y=train['predict'], 
                        mode='markers', name='Predicted value',
                        marker=dict(color='orange')))
fig.update_layout(
    title="Training set"
    )

fig.show()

In [15]:

def calculate_prediction_errors(input_data):
    return (abs(input_data['value'] -input_data['predict'])).to_numpy()

train_pred_errors = calculate_prediction_errors(train)
valid_pred_errors = calculate_prediction_errors(valid)

STATIC THRESHOLD

In [16]:
pred_error_threshold = np.mean(train_pred_errors) + 3 * np.std(train_pred_errors)


In [17]:
window=40
std_coef=5
train_pred_errors_windowed = pd.Series(train_pred_errors).rolling(window=window, min_periods=1)
# Dynamic threshold for the training data
train_dynamic_threshold = train_pred_errors_windowed.mean() + std_coef * train_pred_errors_windowed.std()

valid_pred_errors_windowed = pd.Series(valid_pred_errors).rolling(window=window, min_periods=1)
# Dynamic threshold for the validation data
valid_dynamic_threshold = valid_pred_errors_windowed.mean() + std_coef * valid_pred_errors_windowed.std()

In [18]:
from sklearn.metrics import precision_recall_fscore_support

def calculate_metrics(ground_truth: pd.DataFrame, anomalies_idxs: list):
    predictions = pd.DataFrame(
        index=range(len(ground_truth)), 
        columns=['predicted']
    )
    predictions['predicted'] = 0
    predictions.iloc[anomalies_idxs] = 1
    
    # Calculation of the confusion matrix can be done using pandas
    confusion_matrix = pd.crosstab(
        ground_truth.loc[:, 'anomaly_label'],
        predictions['predicted'], 
        margins=True
    )
    precision, recall, f1, _ = precision_recall_fscore_support(
        ground_truth.loc[:, 'anomaly_label'],
        predictions['predicted'], 
        beta=2., 
        average='binary'
    )
    return confusion_matrix, precision, recall, f1

DYNAMIC THRESHOLD

In [19]:
def detect_anomalies(pred_error_threshold,df):
    # Calculate errors for the gicen data
    test_reconstruction_errors = calculate_prediction_errors(df)
    # Filter errors w/ the threshold
    predicted_anomalies = list(
        map(lambda v: 1 if v > pred_error_threshold else 0,test_reconstruction_errors)
    )
    df['anomaly_predicted'] = predicted_anomalies
    # Extract indexes of the filtered anomalies
    indexes = [i for i, x in enumerate(predicted_anomalies) if x == 1]
    return indexes

train_anomalies_idxs = detect_anomalies(
    pred_error_threshold, train
)
valid_anomalies_idxs = detect_anomalies(
    pred_error_threshold, valid
)

In [20]:
train_detected=train.iloc[train_anomalies_idxs]

In [21]:
import plotly.graph_objects as go
layout = dict(xaxis=dict(title='Timestamp'), yaxis=dict(title='CPU Utilization')) 

# Create the figure for plotting the data
fig = go.Figure(layout=layout) 

# Add non-anomaly data to the figure
fig.add_trace(go.Scatter(x=train['timestamp'], y=train['value'], 
                         mode='markers', name='Non-anomaly',
                         marker=dict(color='blue')))

# Add anomaly data to the figure
fig.add_trace(go.Scatter(x=train_anomalies['timestamp'],
                         y=train_anomalies['value'], 
                         mode='markers', name='Anomaly',
                         marker=dict(color='green', size=13)))

fig.add_trace(go.Scatter(x=train_detected['timestamp'], y=train_detected['value'], 
                         mode='markers', name='Predicted Anomaly',
                         marker=dict(color='red')))
fig.update_layout(
    title="Training set"
)
fig.show()

In [22]:
valid_detected=valid.iloc[valid_anomalies_idxs]

In [23]:

fig = go.Figure()
fig.add_trace(go.Scatter(x=valid['timestamp'], y=valid['value'], 
                         mode='markers', name='Non-anomaly',
                         marker=dict(color='blue')))
fig.add_trace(go.Scatter(x=valid_anomalies['timestamp'],
                         y=valid_anomalies['value'], 
                         mode='markers', name='Anomaly',
                         marker=dict(color='green', size=13)))

fig.add_trace(go.Scatter(x=valid_detected['timestamp'], y=valid_detected['value'], 
                         mode='markers', name='predicted Anomaly',
                         marker=dict(color='red')))
fig.update_layout(
    title="Validation set"
    )

fig.show()

### METRICS

In [24]:
li=[0]*4032
k=valid_anomalies.index.tolist()
for i in k:
  li[i]=1
valid['predicted']=li

In [25]:
li=[0]*4032
k=train_anomalies.index.tolist()
for i in k:
  li[i]=1
train['predicted']=li

In [26]:
from sklearn.metrics import precision_recall_fscore_support

def calculate_metrics(
    ground_truth: pd.DataFrame, anomalies_idxs: list
    ):
    predictions = pd.DataFrame(
        index=range(len(ground_truth)), 
        columns=['predicted']
    )
    predictions['predicted'] = 0
    predictions.iloc[anomalies_idxs] = 1
    
    anomalies_given=(ground_truth['predicted'].tolist()).count(1)
    # Calculation of the confusion matrix can be done using pandas
    confusion_matrix = pd.crosstab(
        ground_truth.loc[:, 'anomaly_predicted'],
        predictions['predicted'], 
        margins=True
    )
    precision, recall, f1, _ = precision_recall_fscore_support(
        ground_truth.loc[:, 'anomaly_predicted'],
        predictions['predicted'], 
        beta=2., 
        average='binary'
    )
    
    precision=anomalies_given/len(anomalies_idxs)
    
    f1=2*(precision*recall)/(precision+recall)
    return confusion_matrix, precision, recall, f1

In [27]:
train_conf_matrix, *train_metrics = calculate_metrics(
    train, train_anomalies_idxs
)

# Pretty printing of the metrics
print(f'Train:\n Precision: {train_metrics[0]:.3f}\n' 
      f'Recall: {train_metrics[1]:.3f}\n' 
      f'F1 score: {train_metrics[2]:.3f}')

Train:
 Precision: 0.044
Recall: 1.000
F1 score: 0.085


In [28]:

valid_conf_matrix, *valid_metrics = calculate_metrics(
    valid, valid_anomalies_idxs
)

print(f'Valid:\n Precision: {valid_metrics[0]:.3f}\n' 
      f'Recall: {valid_metrics[1]:.3f}\n' 
      f'F1 score: {valid_metrics[2]:.3f}')

Valid:
 Precision: 0.018
Recall: 1.000
F1 score: 0.035
